# Function calling with the `openai` Python library

This notebook demonstrates how to create a reusable `query_llm` function that allows calling the OpenAI Chat Completions endpoint with optional arbitrary function-calling arguments—to return either structured data or unstructured text.

Note that OpenAI function calling is designed to work with Pydantic models. You specify a Pydantic model to describe the structured data you want, and the function will return JSON corresponding to that model. You can then convert the JSON to an instance of the model using the Pydantic model's `.model_validate()` method.

Note that we also extend the OpenAI client to allow for specifying the LLM `model` and `fallback` model up front. This allows for the model to be specified when initializing the client, rather than every time we call the LLM. We also add a `total_cost` attribute to allow for tracking the cumulative cost of our API calls.

In [18]:
from os import getenv
from pathlib import Path
from typing import Optional, Literal
import json
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from openai import OpenAI, BadRequestError
from openai.types.chat import ChatCompletion

# Data structure for LLM project map response (use Literal rather than Enum
# because it plays nicer with LLM prompting and JSON serialization
class FileClassification(BaseModel):
    path: Path = Field(description="file path relative to the project root")
    role: Literal["source", "configuration", "build or deployment", "documentation", "testing", "database", "utility scripts", "assets or data", "specialized"] = Field(
            default=None, description="role the file plays in the project"
        )

# Data structure for a list of FileClassifications
class FileClassificationList(BaseModel):
    files: list[FileClassification] = Field(
            description="List of file classifications"
        )

    # Method to convert a FileClassificationList to a JSON-formatted string
    def to_json(self) -> str:
        data_dict = self.model_dump(exclude_unset=True)

        # Convert Path objects to str
        for file in data_dict.get("files", []):
            file["path"] = str(object=file["path"])

        return json.dumps(obj=data_dict, indent=4)

# Load environment variables from .env
load_dotenv()

# Extend the OpenAI class to include, total_cost, model, and fallback attributes
# (total_cost is used to track the total cost of queries, model is the model used,
# and fallback is the model to use if the context window is exceeded)
class ExtendedOpenAI(OpenAI):
    def __init__(self, *args, model="gpt-3.5-turbo", fallback="gpt-4-turbo", **kwargs):
        super().__init__(*args, **kwargs)
        self.total_cost = 0.0
        self.model = model
        self.fallback = fallback

# Create an ExtendedOpenAI client instance
client = ExtendedOpenAI(
  api_key=getenv('OPENAI_API_KEY')
)

# Models and their costs
models = [
    {
        "name": "gpt-3.5-turbo",
        "max_tokens": 16385,
        "prompt_cost_per_token": 0.5 / 1000000,
        "completion_cost_per_token": 1.5 / 1000000,
    },
    {
        "name": "gpt-4-turbo",
        "max_tokens": 128000,
        "prompt_cost_per_token": 10 / 1000000,
        "completion_cost_per_token": 30 / 1000000,
    },
    {
        "name": "gpt-4",
        "max_tokens": 8192,
        "prompt_cost_per_token": 30 / 1000000,
        "completion_cost_per_token": 60 / 1000000,
    },
    {
        "name": "gpt-4-32k",
        "max_tokens": 32768,
        "prompt_cost_per_token": 30 / 1000000,
        "completion_cost_per_token": 0.12 / 1000000,
    }
]

# Functions to enforce structured output from the chatbot
functions = [
        {
          "name": "classify_project_files_by_role",
          "description": "Identify the role that each file plays in a software project",
          "parameters": FileClassificationList.model_json_schema()
        }
    ]

# Prompt template for determining the roles that files play in the project
file_classification_prompt = (
    "We have mapped the file structure of a project folder for an existing "
    "coding project. Based solely on the file structure, let's attempt to "
    "classify them by the role they play in the project. We will label code "
    "modules, entry points, and endpoints as 'source'; config files, "
    "environment files, and dependency files as 'configuration'; build files, "
    "Docker files, and CI/CD files as 'build or deployment'; READMEs, "
    "CHANGELOGs, pseudocodes, project maps, licenses, and docs as "
    "'documentation'; unit tests as 'testing'; migration, schema, and seed "
    "files as 'database', utility and action scripts as 'utility scripts', "
    "static assets like images, CSS, CSV, and JSON files as 'assets and "
    "data', and anything else that doesn't fit these categories (e.g., "
    "compiled distribution files) as 'specialized'.\n"
    "Here is the map of the project file structure:\n%s"
)

def classify_with_openai(input_str: str) -> FileClassificationList:
    # Query the LLM to update the project map
    project_map: list[dict[str]] = query_llm(
                prompt=file_classification_prompt % input_str,
                functions=functions
            )

    # Create a FileClassificationList from the project map
    print(project_map.choices[0])
    json_project_map: str = json.loads(s=project_map.choices[0].message.function_call.arguments)
    parsed_project_map: FileClassificationList = FileClassificationList.model_validate(obj=json_project_map)

    # Return the project map
    return parsed_project_map


# Get max_tokens based on model_name
def get_max_tokens(long: bool = False) -> int:
    # Set model_name based on config + `long` argument
    model_name = client.model if long else client.fallback
    
    # Set max_tokens based on model_name
    max_tokens = [model['max_tokens'] for model in models if model['name'] in model_name][0]

    # Return the chatbot instance
    return max_tokens


# Calculate cost of a query
def calculate_cost(prompt_tokens: int, completion_tokens: int, model_used: str) -> float:
    # Get cost per token for the model used from the models object
    prompt_cost_per_token = [model['prompt_cost_per_token'] for model in models if model['name'] == model_used][0]
    completion_cost_per_token = [model['completion_cost_per_token'] for model in models if model['name'] == model_used][0]
    
    # Calculcate and return the total cost of the query
    total_cost = (prompt_tokens * prompt_cost_per_token) + (completion_tokens * completion_cost_per_token)
    return total_cost


# Query a chatbot using a prompt, and optionally a functions list
def query_llm(prompt: str, functions: Optional[list[dict]] = None) -> str:
    # Generate the output from the input
    try:
        model_used: str = client.model
        response: ChatCompletion = client.chat.completions.create(
            model=client.model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            functions=functions
        )
    except BadRequestError as e:
        # If we exceed context limit, check if long_context_fallback is None
        if client.fallback is None:
            # If long_context_fallback is None, raise the error
            raise e
        else:
            # If long_context_fallback is not None, warn and use long_context_fallback
            print("Encountered error:\n" + e + "\nTrying again with long_context_fallback.")
            model_used: str = client.fallback
            response: ChatCompletion = client.chat.completions.create(
                model=client.fallback,
                messages=[
                    {"role": "user", "content": prompt}
                ],
                functions=functions
            )
    
    # Update the total cost
    client.total_cost += calculate_cost(
            prompt_tokens=response.usage.prompt_tokens,
            completion_tokens=response.usage.completion_tokens,
            model_used=model_used
        )

    # Return the response object
    return response

sample_project_map: dict[list[dict]] = {"files": [
    {"path": ".gitignore", "role": None},
    {"path": "poetry.lock", "role": None},
    {"path": "pyproject.toml", "role": None},
    {"path": "README.md", "role": None},
    {"path": "LICENSE", "role": None},
    {"path": "tests\\__init__.py", "role": None},
    {"path": "tests\\test_file_handler.py", "role": None},
    {"path": "file_handler\\file_handler.py", "role": None},
]}

# Convert the file list to a string
input_str: str = ", ".join([item["path"] for item in sample_project_map["files"]])

# Classify the project files by role
file_classifications: FileClassificationList = classify_with_openai(input_str=input_str)

# Print the file classifications
print(file_classifications.to_json())


Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"files":[{"path":".gitignore","role":"configuration"},{"path":"poetry.lock","role":"configuration"},{"path":"pyproject.toml","role":"configuration"},{"path":"README.md","role":"documentation"},{"path":"LICENSE","role":"documentation"},{"path":"tests/__init__.py","role":"testing"},{"path":"tests/test_file_handler.py","role":"testing"},{"path":"file_handler/file_handler.py","role":"source"}]}', name='classify_project_files_by_role'), tool_calls=None))
{
    "files": [
        {
            "path": ".gitignore",
            "role": "configuration"
        },
        {
            "path": "poetry.lock",
            "role": "configuration"
        },
        {
            "path": "pyproject.toml",
            "role": "configuration"
        },
        {
            "path": "README.md",
            "role": "documentation"
        }

Note that we wrote our `query_llm` function in such a way that it can flexibly be used for calling arbitrary functions, as above, or for interacting with the LLM in the more usual way with text queries, as below.

In [19]:
# Prompt to generate a pseudocode summary of a code module
pseudocode_prompt = (
    "Generate an abbreviated natural-language pseudocode summary of the "
    "following code. Make sure to include function, class, and argument names "
    "and to indicate where objects are imported from so a reader can "
    "understand the execution context and usage. Well-formatted pseudocode "
    "will separate object and function blocks with a blank line and will use "
    "hierarchical ordered and unordered lists to show execution sequence and "
    "logical relationships.\nHere is the code to summarize:\n%s"
)

sample_code = '''
def get_fibonacci_numbers_below_n(n) -> list[int]:
  fibonacci_sequence = [1, 2]
  while fibonacci_sequence[len(fibonacci_sequence)-1] < n:
    fibonacci_sequence.append(sum(
        fibonacci_sequence[(len(fibonacci_sequence) - 2):len(fibonacci_sequence)]
      ))
  return fibonacci_sequence
  '''

def summarize_with_openai(input_str: str) -> str:
    # Generate a prompt for the chatbot
    prompt = pseudocode_prompt % sample_code

    # Query the chatbot for a summary and parse the output
    generated_summary: ChatCompletion = query_llm(
                prompt=prompt
            )
    
    return generated_summary.choices[0].message.content

# Generate a pseudocode summary of the sample code
pseudocode_summary: str = summarize_with_openai(input_str=sample_code)

# Print the pseudocode summary
print(pseudocode_summary)

Function: 
- Name: get_fibonacci_numbers_below_n
- Argument: 
  - n (int)
- Return: 
  - list[int]

---

1. Start
2. Create a list called fibonacci_sequence with initial values [1, 2]
3. While the last element in fibonacci_sequence is less than n:
    1. Append the sum of the last two elements in fibonacci_sequence to the list
4. Return fibonacci_sequence

---
